In [30]:
import requests
import os
import pandas as pd
from datetime import date, datetime, timedelta
import time
from io import StringIO
idag = date.today().isoformat()[0:10]
tokenCanvas = os.environ['tokenCanvas']

headers = {'Authorization': f'Bearer {tokenCanvas}',}
params = {'per_page': 100,}

# Initialisering

Først må eg lese fila over alle som er aktive lærarar i Canvas, og sjekke om dei er medlem i eit testemne. Dette vert berre gjort ein gang, og resultatet vert lagra i ei fil. 


In [3]:
data = pd.read_csv("users_filtered.csv")
data['brukarnamn'] = data['login_id'].apply(lambda x: x.split('@')[0])
eksisterande = data[data['brukarnamn'].str.isalpha()]
eksisterande.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1475 entries, 182 to 18702
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   user_id         1475 non-null   object 
 1   integration_id  0 non-null      float64
 2   login_id        1475 non-null   object 
 3   first_name      1475 non-null   object 
 4   last_name       1475 non-null   object 
 5   short_name      0 non-null      float64
 6   email           1475 non-null   object 
 7   status          1475 non-null   object 
 8   brukarnamn      1475 non-null   object 
dtypes: float64(2), object(7)
memory usage: 115.2+ KB


Den neste kodesnutten lager ei liste over alle lærarar, og kva testemne dei eventuelt er med i.

Køyrt og rensa 250526 - det skal ikkje vere nødvendig å køyre den fleire ganger. Resultatet etter vasking er lagra i `250526 oversikt.csv`.

In [3]:
# for person in eksisterande['login_id']:
#     print(f"{person} | ", end="")
#     try:
#         params = {'per_page': 100}
#         url = f"https://hvl.instructure.com/api/v1/accounts/1/users?search_term={person}"
#         respons = requests.get(url, headers=headers, params=params)
#         user_id = respons.json()[0]['id']
#         name = respons.json()[0]['name']
#         print(f"{user_id} | {name} | ", end="")
#         emneliste = []
#         params = {'per_page': 100, 'state': ['unpublished']}
#         url = f"https://hvl.instructure.com/api/v1/users/{user_id}/courses"
#         respons = requests.get(url, headers=headers, params=params)
#         for emne in respons.json():
#             emneliste.append(emne['name'])
#         params = {'per_page': 100, 'state': ['available']}
#         url = f"https://hvl.instructure.com/api/v1/users/{user_id}/courses"
#         respons = requests.get(url, headers=headers, params=params)
#         for emne in respons.json():
#             emneliste.append(emne['name'])
#         for e in emneliste: 
#             if 'Testemne' in e:
#                 print(f"{e}, ", end=" ")
#         print("")
#     except:
#         print("Ikkje lærar.") 

## Opprette testemne for alle lærarar som ikkje har det.

Eg må gå gjennom fila frå forrige blokk og plukke ut alle som har "Lag nytt emne" i Kontrollfelt:


In [26]:
data = pd.read_csv("250526 oversikt.csv")
data['lag_nytt'] = data.apply(lambda row: str(row['namn']) in str(row['testemne']), axis=1)
print(f"Skal lage testemne for {len(data[data['lag_nytt']==False])} personar:")
for i, r in data[data['lag_nytt']==False].iterrows():
    print([r['brukar'],r['id'],r['namn']])
data.info()

Skal lage testemne for 53 personar:
['gvb@hvl.no', 1501, 'Graziella Van den Bergh']
['hebo@hvl.no', 13429, 'Heine Bøe']
['hdhu@hvl.no', 75662, 'Hariom Dhungana']
['thbo@hvl.no', 89697, 'Therese Bøyum']
['lflo@hvl.no', 31748, 'Lene Ferstad-Løland']
['aknga@hvl.no', 89703, 'Allan Kevin Nganga']
['mtso@hvl.no', 77943, 'Maria Tsouri']
['sihau@hvl.no', 78706, 'Sigrid Haukanes']
['anne@hvl.no', 78081, 'Andreas Rande Nesheim']
['jmto@hvl.no', 98125, 'Janne-Merete Torset Øien']
['nome@hvl.no', 89733, 'Nils Olav Mevatne']
['sumub@hvl.no', 88528, 'Susan Kyomuhendo Munabi-Babigumira']
['mbdp@hvl.no', 88805, 'Mario Blazquez de Paz']
['simondev@hvl.no', 4695, 'Simon Daniel de Villiers']
['jobre@hvl.no', 84486, 'John Olav Hauglum Brekke']
['fsve@hvl.no', 89621, 'Frithiof Svenson']
['amj@hvl.no', 95417, 'Anja Møgelvang Jacobsen']
['arsme@hvl.no', 95751, 'Arlene Velasco Smenes']
['alvi@hvl.no', 107900000000001415, 'Aleksander Vie']
['atkha@hvl.no', 12574, 'Anne Therese Kjellevold Hatle']
['arha@hvl.no

Dette er alt for mange, så eg går gjennom lista manuelt og luker ut alle som har ulikt namn:

In [29]:
nye = pd.read_csv("ny.csv")
nye

,user,id,name
0,hdhu@hvl.no,75662,Hariom Dhungana
1,thbo@hvl.no,89697,Therese Bøyum
2,lflo@hvl.no,31748,Lene Ferstad-Løland
3,aknga@hvl.no,89703,Allan Kevin Nganga
4,mtso@hvl.no,77943,Maria Tsouri
5,sihau@hvl.no,78706,Sigrid Haukanes
6,anne@hvl.no,78081,Andreas Rande Nesheim
7,jmto@hvl.no,98125,Janne-Merete Torset Øien
8,fsve@hvl.no,89621,Frithiof Svenson
9,alvi@hvl.no,107900000000001415,Aleksander Vie


Dette er overkommeleg, så no kan eg lage ei rutine som oppretter eit testemne for kvar av desse. Endepunktet er `post /v1/accounts/{account_id}/courses`, og parametrane er
- namn på emnet (f"Testemne for {namn}")
- emnekode ("Testemne")
- sis-id (f"hvlYYMMDD-e{løpenummer}-akv")
- heimeside ("course[default_view]": "syllabus")
- Tekst på heimesida ("Dette er ditt testemne ...")

Når eg har oppretta emnet kan eg legge til brukaren som lærar. Endepunket er `post /v1/courses/{course_id}/enrollments` og parametrane er 
- course_id: denne får eg frå respons.json()['id']
- enrollment[user_id]
- enrollment[type]: TeacherEnrollment
- enrollment[enrollment_state]: active

Og for å fjerne kalenderen frå heimesida må eg oppdatere emnet. Endepunktet er `put /v1/courses/{course_id}/settings` og parametrane er
- course_id: som over
- syllabus_course_summary: false

Til slutt må eg legge læraren inn i lista over dei som alt har testemne, slik at eg slepp å sjekke dette kvar natt.

In [39]:
har_testemne = pd.read_csv("har_testemne.csv")
konto = 9044
løpenummer = 100
for i, r in nye.iterrows():
    url = f"https://hvl.instructure.com/api/v1/accounts/{konto}/courses"
    params = {
        'course[name]': f"Testemne for {r['name']}",
        'course[course_code]': "Testemne", 
        'course[sis_course_id]': f"hvl{idag}-e{løpenummer}-akv",
        'course[default_view]': 'syllabus',
        'course[syllabus_body]': "Dette er ditt testemne; her kan du gjere alt du vil prøve på i eit 'vanleg' emne. Ting du lager her kan du hente inn i andre emne til bruk i undervising.",
        }
    respons = requests.post(url, headers=headers, params=params)
    resultat = respons.json()
    print(resultat)
    id = resultat['id']
    løpenummer += 1
    url = f"https://hvl.instructure.com/api/v1/courses/{id}/enrollments"
    params = {
        'enrollment[user_id]': r['id'],
        'enrollment[type]': "TeacherEnrollment",
        'enrollment[enrollment_state]': 'active',
    }
    respons = requests.post(url, headers=headers, params=params)
    print(respons.json())
    url = f"https://hvl.instructure.com/api/v1/courses/{id}/settings"
    params = {
        'syllabus_course_summary': 'false',
    }
    respons = requests.put(url, headers=headers, params=params)
    har_testemne[len(har_testemne)] = [r['user'], r['id'], r['name']]
har_testemne.to_csv("har_testemne.csv", index = False)


{'id': 33983, 'name': 'Testemne for Hariom Dhungana', 'account_id': 9044, 'uuid': 'UDIhP1WD42lNt7VvX8cHlKIsl4Q1yTepys2TTTeD', 'start_at': None, 'conclude_at': None, 'grading_standard_id': None, 'is_public': None, 'created_at': '2025-05-27T08:04:41Z', 'allow_student_forum_attachments': True, 'course_code': 'Testemne', 'default_view': 'syllabus', 'root_account_id': 1, 'enrollment_term_id': 1, 'open_enrollment': None, 'allow_wiki_comments': None, 'self_enrollment': None, 'license': None, 'restrict_enrollments_to_course_dates': False, 'grade_passback_setting': None, 'end_at': None, 'public_syllabus': False, 'public_syllabus_to_auth': False, 'storage_quota_mb': 524, 'is_public_to_auth_users': False, 'homeroom_course': False, 'course_color': None, 'friendly_name': None, 'hide_final_grades': False, 'apply_assignment_group_weights': False, 'calendar': {'ics': 'https://hvl.instructure.com/feeds/calendars/course_UDIhP1WD42lNt7VvX8cHlKIsl4Q1yTepys2TTTeD.ics'}, 'time_zone': 'Europe/Copenhagen', 'b

# Neste steg: sjekke nye lærarar kvar natt

No må eg lage meg ei kontrolliste med kven som har testemne. Kvar natt sjekker eg så importen av lærarar, og ser om det er nokon der som ikkje er på lista over eksisterande lærarar. Desse skal i så fall få eit testemne etter malen over.

Først må eg lese fila over alle som vert overført i natt frå FS:

In [47]:
har_testemne = pd.read_csv("har_testemne.csv")
url = "https://hvl.instructure.com/api/v1/accounts/1/sis_imports"
svar = requests.get(url, headers=headers, params=params)
importerte_filar = svar.json()

no = datetime.now()
igår = (datetime.now() - timedelta(hours=24)).isoformat()
filliste = []
for i in importerte_filar['sis_imports']:
    import_dato = datetime.fromisoformat(i['created_at'].replace('Z', '+00:00')).replace(tzinfo=None)
    igår_dato = datetime.fromisoformat(igår)
    if (import_dato > igår_dato):
        for j in i['csv_attachments']:
            if j['filename'] == "users_filtered.csv":
                filliste.append({'dato': i['created_at'], 'url': j['url']})
users_liste = []
for i in filliste:
    temp = pd.read_csv(StringIO(requests.get(i['url'], headers=headers).content.decode("utf-8")))
    print(f"{i['dato']}: {len(temp)}")
    users_liste.append(temp)

overført = pd.concat(users_liste)
overført['brukarnamn'] = overført['login_id'].apply(lambda x: x.split('@')[0])
overført = overført[overført['brukarnamn'].str.isalpha()]
nye = overført[~overført['brukarnamn'].isin(har_testemne['brukar'])]
nye

2025-05-27T10:12:47Z: 1
2025-05-27T02:29:12Z: 18665
2025-05-26T15:12:44Z: 1
2025-05-26T14:12:49Z: 3


,user_id,integration_id,login_id,first_name,last_name,short_name,email,status,brukarnamn


Dersom eg ser at det er nye tilsette skal eg lage nye testemner for dei, og eg må legge dei til i fila `har_testemne.csv`: 

Men for å finne brukaren for å legge hen inn i emnet må eg vite `user_id`, som er internt i Canvas. Eg må difor søke opp brukaren ved hjelp av SIS-ID eller noko slikt.

In [ ]:
har_testemne = pd.read_csv("har_testemne.csv")
konto = 9044
løpenummer = 100
for i, r in nye.iterrows():
    # Først lager eg emnet
    namn = f"{r['first_name']} {r['last_name']}"
    url = f"https://hvl.instructure.com/api/v1/accounts/{konto}/courses"
    params = {
        'course[name]': f"Testemne for {namn}",
        'course[course_code]': "Testemne", 
        'course[sis_course_id]': f"hvl{idag}-testemne{løpenummer}-akv",
        'course[default_view]': 'syllabus',
        'course[syllabus_body]': "Dette er ditt testemne; her kan du gjere alt du vil prøve på i eit 'vanleg' emne. Ting du lager her kan du hente inn i andre emne til bruk i undervising.",
        }
    respons = requests.post(url, headers=headers, params=params)
    resultat = respons.json()
    id = resultat['id']

    # Så leiter eg etter canvas_id for brukaren
    sis_id = r['user_id']
    url = f"https://hvl.instructure.com/api/v1/accounts/1/users?search_term={sis_id}"
    user_id = requests.get(url, headers=headers).json()['id']

    # Så kan eg legge brukaren inn som lærar
    url = f"https://hvl.instructure.com/api/v1/courses/{id}/enrollments"
    params = {
        'enrollment[user_id]': user_id,
        'enrollment[type]': "TeacherEnrollment",
        'enrollment[enrollment_state]': 'active',
    }
    respons = requests.post(url, headers=headers, params=params)

    # Til slutt slår eg av vising av kalenderen
    url = f"https://hvl.instructure.com/api/v1/courses/{id}/settings"
    params = {
        'syllabus_course_summary': 'false',
    }
    respons = requests.put(url, headers=headers, params=params)

    # Og så gjenstår det berre å legge brukaren til i lista over dei som har eit testemne
    har_testemne[len(har_testemne)] = [r['user'], r['id'], r['name']]
    løpenummer += 1

# Og skrive dette til fil.
har_testemne.to_csv("har_testemne.csv", index = False)